# Test 6 (paper) - Experimento 6 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 6"

### Hiperparametros
* **epoch = 120**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1024 x 800**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = true**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1752
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  67789 Sep 16 23:55 master-test_11.ipynb
-rw-r--r-- 1 root root  88232 Sep 17 07:27 master-test_11_fase_2-Copy1.ipynb
-rw-r--r-- 1 root root  67792 Sep 17 08:36 master-test_11_fase_2.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 23:44 master-test_12.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 22:02 master-test_12_fase_2.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 master-test_5-fase_2.ipynb
-rw-r--r-- 1 root root 350144 Sep 17 08:42 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root 111017 Sep 18 16:23 master-test_6.ipynb
-rw-r--r-- 1 root root 270148 Sep 17 16:09 master-test_6_fase_2.ipynb
drwxr-xr-x 4 root root   4096 Sep 18 19:33 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 02:27 old
-rw-r--r-- 1 root root 163924 Sep 18 23:12 test_5_cor

# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa


# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [3]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "arandano-pinton")
        #self.add_class("dataset", 4, "arandano-semi")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 11074
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            if is_train and int(image_id) >= 41590 :
                #print("image_id: ", image_id)
                continue
             
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 11074
            if not is_train and int(image_id) < 41590:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
                        
            #quita todas las imagenes no etiquetadas
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
         # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        ## se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.
        
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [4]:
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_6_fase_2_2_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    
    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 
    
    MINI_MASK_SHAPE = (56, 56)
    
    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [5]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [6]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_fase_2_20220913T0749/mask_rcnn_arandano_cfg_test_5_fase_2__0300.h5'
#pesos = 'arandano_cfg_test_6_2_20220918T1934/mask_rcnn_arandano_cfg_6_2__0100.h5'
pesos = 'arandano_cfg_test_5_fase_2_2_20220919T0845/mask_rcnn_arandano_cfg_test_5_fase_2_2__0120.h5'

conjunto_datos = 'customImages/test_6_fase_2'


In [7]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

# iniciar entrenamiento del modelo
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, custom_callbacks = [tensorboard_callback], epochs=120, layers='all', augmentation=augmentation)

model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=120, layers='all', augmentation=augmentation)



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_6_fase_2_2_20220920T0813/mask_rcnn_arandano_cfg_test_6_fase_2_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b       

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/120


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_5:0", shape=(Non

146/146 [==============================] - 830s 6s/step - batch: 72.5000 - size: 2.0000 - loss: 4.1938 - rpn_class_loss: 1.0837 - rpn_bbox_loss: 1.5021 - mrcnn_class_loss: 0.3823 - mrcnn_bbox_loss: 0.7208 - mrcnn_mask_loss: 0.5049 - val_loss: 2.8577 - val_rpn_class_loss: 0.6160 - val_rpn_bbox_loss: 0.8030 - val_mrcnn_class_loss: 0.3909 - val_mrcnn_bbox_loss: 0.5003 - val_mrcnn_mask_loss: 0.5475
Epoch 2/120
146/146 [==============================] - 637s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 2.1837 - rpn_class_loss: 0.3520 - rpn_bbox_loss: 0.4873 - mrcnn_class_loss: 0.4150 - mrcnn_bbox_loss: 0.4569 - mrcnn_mask_loss: 0.4726 - val_loss: 2.7458 - val_rpn_class_loss: 0.4926 - val_rpn_bbox_loss: 0.6684 - val_mrcnn_class_loss: 0.6538 - val_mrcnn_bbox_loss: 0.4571 - val_mrcnn_mask_loss: 0.4740
Epoch 3/120
146/146 [==============================] - 766s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 2.0170 - rpn_class_loss: 0.2961 - rpn_bbox_loss: 0.4208 - mrcnn_class_loss: 0.4507 -

Epoch 21/120
146/146 [==============================] - 780s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 1.3117 - rpn_class_loss: 0.1530 - rpn_bbox_loss: 0.2534 - mrcnn_class_loss: 0.3360 - mrcnn_bbox_loss: 0.2302 - mrcnn_mask_loss: 0.3391 - val_loss: 2.3209 - val_rpn_class_loss: 0.4175 - val_rpn_bbox_loss: 0.6121 - val_mrcnn_class_loss: 0.5274 - val_mrcnn_bbox_loss: 0.3280 - val_mrcnn_mask_loss: 0.4359
Epoch 22/120
146/146 [==============================] - 767s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 1.3212 - rpn_class_loss: 0.1494 - rpn_bbox_loss: 0.2548 - mrcnn_class_loss: 0.3482 - mrcnn_bbox_loss: 0.2302 - mrcnn_mask_loss: 0.3387 - val_loss: 2.1935 - val_rpn_class_loss: 0.4542 - val_rpn_bbox_loss: 0.5784 - val_mrcnn_class_loss: 0.4794 - val_mrcnn_bbox_loss: 0.3097 - val_mrcnn_mask_loss: 0.3719
Epoch 23/120
146/146 [==============================] - 606s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 1.2842 - rpn_class_loss: 0.1406 - rpn_bbox_loss: 0.2458 - mrcnn_class

Epoch 41/120
146/146 [==============================] - 707s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 1.0713 - rpn_class_loss: 0.1125 - rpn_bbox_loss: 0.1998 - mrcnn_class_loss: 0.2818 - mrcnn_bbox_loss: 0.1803 - mrcnn_mask_loss: 0.2969 - val_loss: 2.2053 - val_rpn_class_loss: 0.4903 - val_rpn_bbox_loss: 0.4804 - val_mrcnn_class_loss: 0.5815 - val_mrcnn_bbox_loss: 0.2964 - val_mrcnn_mask_loss: 0.3566
Epoch 42/120
146/146 [==============================] - 739s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 1.0650 - rpn_class_loss: 0.1135 - rpn_bbox_loss: 0.1976 - mrcnn_class_loss: 0.2738 - mrcnn_bbox_loss: 0.1807 - mrcnn_mask_loss: 0.2994 - val_loss: 1.9103 - val_rpn_class_loss: 0.3190 - val_rpn_bbox_loss: 0.4669 - val_mrcnn_class_loss: 0.5211 - val_mrcnn_bbox_loss: 0.2700 - val_mrcnn_mask_loss: 0.3332
Epoch 43/120
146/146 [==============================] - 788s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 1.0455 - rpn_class_loss: 0.1071 - rpn_bbox_loss: 0.1925 - mrcnn_class

Epoch 61/120
146/146 [==============================] - 628s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.9311 - rpn_class_loss: 0.0887 - rpn_bbox_loss: 0.1673 - mrcnn_class_loss: 0.2419 - mrcnn_bbox_loss: 0.1553 - mrcnn_mask_loss: 0.2780 - val_loss: 1.8587 - val_rpn_class_loss: 0.3816 - val_rpn_bbox_loss: 0.4044 - val_mrcnn_class_loss: 0.4701 - val_mrcnn_bbox_loss: 0.2741 - val_mrcnn_mask_loss: 0.3285
Epoch 62/120
146/146 [==============================] - 672s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.8459 - rpn_class_loss: 0.0750 - rpn_bbox_loss: 0.1472 - mrcnn_class_loss: 0.2141 - mrcnn_bbox_loss: 0.1401 - mrcnn_mask_loss: 0.2694 - val_loss: 2.0785 - val_rpn_class_loss: 0.4702 - val_rpn_bbox_loss: 0.4583 - val_mrcnn_class_loss: 0.5431 - val_mrcnn_bbox_loss: 0.2788 - val_mrcnn_mask_loss: 0.3281
Epoch 63/120
146/146 [==============================] - 702s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.8751 - rpn_class_loss: 0.0809 - rpn_bbox_loss: 0.1584 - mrcnn_class

Epoch 81/120
146/146 [==============================] - 797s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7427 - rpn_class_loss: 0.0594 - rpn_bbox_loss: 0.1226 - mrcnn_class_loss: 0.1925 - mrcnn_bbox_loss: 0.1173 - mrcnn_mask_loss: 0.2509 - val_loss: 2.2968 - val_rpn_class_loss: 0.6067 - val_rpn_bbox_loss: 0.4673 - val_mrcnn_class_loss: 0.6331 - val_mrcnn_bbox_loss: 0.2652 - val_mrcnn_mask_loss: 0.3245
Epoch 82/120
146/146 [==============================] - 732s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7708 - rpn_class_loss: 0.0645 - rpn_bbox_loss: 0.1340 - mrcnn_class_loss: 0.1965 - mrcnn_bbox_loss: 0.1246 - mrcnn_mask_loss: 0.2511 - val_loss: 2.0445 - val_rpn_class_loss: 0.4905 - val_rpn_bbox_loss: 0.4658 - val_mrcnn_class_loss: 0.5160 - val_mrcnn_bbox_loss: 0.2568 - val_mrcnn_mask_loss: 0.3154
Epoch 83/120
146/146 [==============================] - 726s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7889 - rpn_class_loss: 0.0697 - rpn_bbox_loss: 0.1417 - mrcnn_class

Epoch 101/120
146/146 [==============================] - 757s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6716 - rpn_class_loss: 0.0506 - rpn_bbox_loss: 0.1114 - mrcnn_class_loss: 0.1660 - mrcnn_bbox_loss: 0.1065 - mrcnn_mask_loss: 0.2371 - val_loss: 2.4364 - val_rpn_class_loss: 0.7507 - val_rpn_bbox_loss: 0.5652 - val_mrcnn_class_loss: 0.5545 - val_mrcnn_bbox_loss: 0.2348 - val_mrcnn_mask_loss: 0.3311
Epoch 102/120
146/146 [==============================] - 825s 6s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6615 - rpn_class_loss: 0.0513 - rpn_bbox_loss: 0.1163 - mrcnn_class_loss: 0.1592 - mrcnn_bbox_loss: 0.1032 - mrcnn_mask_loss: 0.2315 - val_loss: 2.1888 - val_rpn_class_loss: 0.5713 - val_rpn_bbox_loss: 0.4568 - val_mrcnn_class_loss: 0.5848 - val_mrcnn_bbox_loss: 0.2561 - val_mrcnn_mask_loss: 0.3198
Epoch 103/120
146/146 [==============================] - 637s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6364 - rpn_class_loss: 0.0430 - rpn_bbox_loss: 0.1018 - mrcnn_cl